In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('input/train.csv')
df.head()

## Cleaning Columns

In [ ]:
df.columns[df.isnull().sum() > 0]

In [ ]:
df.fillna(method='bfill',inplace=True)

In [ ]:
df.columns[df.isnull().sum() > 0]

In [ ]:
df.fillna(method='ffill',inplace=True)

In [ ]:
df.columns[df.isnull().sum() > 0]

In [ ]:
# finding numeric and categorical columns
Numeric_columns=df.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df.columns).difference(set(Numeric_columns))
# numeric cols to numeric
print(categorical_col)

In [ ]:
df['edjefe'] = df['edjefe'].replace({'no': 0, 'yes':1}).astype(float)
df['edjefa'] = df['edjefa'].replace({'no': 0, 'yes':1}).astype(float)

In [ ]:
# finding numeric and categorical columns
Numeric_columns=df.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df.columns).difference(set(Numeric_columns))
# numeric cols to numeric
print(categorical_col)

In [ ]:
df['edjefe'].unique()

In [ ]:
df['edjefa'].unique()

In [ ]:
df['edjefe'].value_counts()

In [ ]:
df['dependency'] = np.sqrt(df['SQBdependency'])

In [ ]:
df.isna().sum().sum()

In [ ]:
col_drops = ['Id','idhogar']
df.drop(col_drops,axis=1,inplace=True)

In [ ]:
# finding numeric and categorical columns
Numeric_columns=df.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df.columns).difference(set(Numeric_columns))
# numeric cols to numeric
print(categorical_col)

## increasing features

In [ ]:
train_set = df

In [ ]:
df['rent_per_adult'] = df['v2a1']/df['hogar_adul']
df['rent_per_person'] = df['v2a1']/df['hhsize']

df['overcrowding_room_and_bedroom'] = (df['hacdor'] + df['hacapo'])/2

df['no_appliances'] = df['refrig'] + df['computer'] + df['television']

df['r4h1_percent_in_male'] = df['r4h1'] / df['r4h3']
df['r4m1_percent_in_female'] = df['r4m1'] / df['r4m3']
df['r4h1_percent_in_total'] = df['r4h1'] / df['hhsize']
df['r4m1_percent_in_total'] = df['r4m1'] / df['hhsize']
df['r4t1_percent_in_total'] = df['r4t1'] / df['hhsize']

df['adult'] = df['hogar_adul'] - df['hogar_mayor']
df['dependency_count'] = train_set['hogar_nin'] + df['hogar_mayor']
df['dependency'] = df['dependency_count'] / df['adult']
df['child_percent'] = df['hogar_nin']/df['hogar_total']
df['elder_percent'] = df['hogar_mayor']/df['hogar_total']
df['adult_percent'] = df['hogar_adul']/df['hogar_total']

df['rent_per_bedroom'] = df['v2a1']/df['bedrooms']
df['adults_per_bedroom'] = df['adult']/df['bedrooms']
df['child_per_bedroom'] = df['hogar_nin']/df['bedrooms']
df['male_per_bedroom'] = df['r4h3']/df['bedrooms']

df['female_per_bedroom'] = df['r4m3']/df['bedrooms']
df['bedrooms_per_person_household'] = df['hhsize']/df['bedrooms']

df['tablet_per_person_household'] = df['v18q1']/df['hhsize']
df['phone_per_person_household'] = df['qmobilephone']/df['hhsize']

df['age_12_19'] = df['hogar_nin'] - df['r4t1']

df['rent_per_room'] = df['v2a1']/df['rooms']
df['bedroom_per_room'] = df['bedrooms']/df['rooms']
df['elder_per_room'] = df['hogar_mayor']/df['rooms']
df['adults_per_room'] = df['adult']/df['rooms']
df['child_per_room'] = df['hogar_nin']/df['rooms']
df['male_per_room'] = df['r4h3']/df['rooms']
df['female_per_room'] = df['r4m3']/df['rooms']
df['room_per_person_household'] = df['hhsize']/df['rooms']
df['escolari_age'] = df['escolari']/df['age']

df['rez_esc_escolari'] = df['rez_esc']/df['escolari']
df['rez_esc_r4t1'] = df['rez_esc']/df['r4t1']
df['rez_esc_r4t2'] = df['rez_esc']/df['r4t2']
df['rez_esc_r4t3'] = df['rez_esc']/df['r4t3']
df['rez_esc_age'] = df['rez_esc']/df['age']


In [ ]:
!pip install xgboost

In [ ]:
!pip install lightgbm

In [ ]:
!pip install bayesian-optimization

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [ ]:
x_1 = df.drop('Target',axis=1)

In [ ]:
df.shape

In [ ]:
x_1.shape

In [ ]:
y = df['Target']

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate,cross_val_score

## Optimizing Xgboost

In [ ]:
def xgb_cv(n_estimators, max_depth, gamma, subsample, data, targets):
    estimator = XGBClassifier(
        n_estimators=n_estimators,
        max_depth = max_depth,
        gamma = gamma,
        # min_child_weight=min_child_weight,
        subsample = subsample,
        random_state = 2,
    )
    cval = cross_val_score(estimator, data, targets
                          , cv=5)
    return cval.mean()

In [ ]:
def optimize_xgb(data, targets):
    def xgb_crossval(n_estimators, max_depth, gamma, subsample):
        return xgb_cv(
            n_estimators=int(n_estimators),
            max_depth = int(max_depth),
            gamma = gamma,
            # min_child_weight=min_child_weight,
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        pbounds={
            "n_estimators": (100, 500),
            "max_depth": (6,15),
            "gamma": (0,10),
            # "min_child_weight": (0,10),
            "subsample": (0.8,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=10)

    print("Final result:", optimizer.max)

In [ ]:
print("--- Optimizing XGBoost ---")
optimize_xgb(x_1, y)

## Optimizing Catboost

In [ ]:
def cb_cv(n_estimators, depth,data, targets):
    estimator = CatBoostClassifier(
        n_estimators=n_estimators,
#         learning_rate=learning_rate,
        depth=depth,
        random_state = 2,
        verbose = 0,
    )
    cval = cross_val_score(estimator, data, targets,
                            cv=5)
    return cval.mean()

In [ ]:
def optimize_cb(data, targets):
    def cb_crossval(n_estimators, depth):
        return cb_cv(
            n_estimators=int(n_estimators),
#             learning_rate = learning_rate,
            depth = int(depth),
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=cb_crossval,
        pbounds={
            "n_estimators": (200, 600),
#             "learning_rate": (0.01,10),
            "depth": (4,16),
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)


In [ ]:
print("--- Optimizing Catboost ---")
optimize_cb(x_1, y)

## Optimizing LGBM

In [ ]:
def lgb_cv(n_estimators, num_leaves, min_child_samples, subsample, data, targets):
    estimator = LGBMClassifier(
        n_estimators=n_estimators,
        num_leaves = num_leaves,
        min_child_samples=min_child_samples,
        subsample = subsample,
        random_state = 2
    )
    cval = cross_val_score(estimator, data, targets, cv=5)
    return cval.mean()

In [ ]:
def optimize_lgb(data, targets):
    def lgb_crossval(n_estimators, num_leaves, min_child_samples, subsample):
        return lgb_cv(
            n_estimators=int(n_estimators),
            num_leaves = int(num_leaves),
            min_child_samples=int(min_child_samples),
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=lgb_crossval,
        pbounds={
            "n_estimators": (100,500),
            "num_leaves": (30,80),
            "min_child_samples": (5,30),
            "subsample": (0.6,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [ ]:
print("--- Optimizing Light GBM ---")
optimize_lgb(x_1, y)

## Model Fitting

In [ ]:
Xg=XGBClassifier()
Lgbm=LGBMClassifier(n_estimators=100,learning_rate=0.1,random_state=42,num_leaves=200)
    
Cataboost=CatBoostClassifier(depth=int(9.253),n_estimators=int(514.1))
estimators = [('Cataboost', Cataboost), ('Xg', Xg), ('Lgbm', Lgbm)]

clf = StackingClassifier(estimators=estimators)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_1,y,random_state=42)

In [ ]:
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_test,y_test)